In [5]:
#default_exp app_2

In [6]:
#export
import json
import plotly
import pandas as pd

from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize

from flask import Flask
from flask import render_template, request, jsonify
from plotly.graph_objs import Bar, Heatmap
#from sklearn.externals import joblib
import joblib
from sqlalchemy import create_engine

In [7]:
#export
app = Flask(__name__)

def tokenize(text):
    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()

    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens



In [8]:
#export
# load data
engine = create_engine('sqlite:///../data/DisasterResponse.db')
df = pd.read_sql_table('tweets', engine)

In [9]:
df.head()

,id,message,original,genre,categories,related,request,offer,aid_related,medical_help,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,related-1;request-0;offer-0;aid_related-0;medi...,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,related-1;request-0;offer-0;aid_related-1;medi...,1,0,0,1,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,related-1;request-0;offer-0;aid_related-0;medi...,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,related-1;request-1;offer-0;aid_related-1;medi...,1,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,related-1;request-0;offer-0;aid_related-0;medi...,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [10]:
#export
# load model
model = joblib.load("../models/classifier.pkl")


# index webpage displays cool visuals and receives user input text for model
@app.route('/')
@app.route('/index')
def index():
    
    # extract data needed for visuals
    # TODO: Below is an example - modify to extract data for your own visuals
    genre_counts = df.groupby('genre').count()['message']
    genre_names = list(genre_counts.index)
    # Creating counts by each labels
    y = df.drop(['id','categories','message','original','genre'], axis=1)
    y0=y.melt()
    total_counts = y0.groupby('variable').sum()['value']
    total_names = list(total_counts.index)
    # Instead of doing correlation, we will be doing Baye's metrics
    # Essention it given a label , how many times other label has occured
    # This gives joint probabilty for, say given a tweet is marked Shelter, what is the probablity it is also marked as food.
    # This is useful as ML algorithms may not label the Shelter tweet also as Food but a human may. 
    cr=y.corr()
    cols=cr.columns.tolist()
    for a,col in enumerate(cr.columns):  
        for i, row in enumerate(cr[col]):
            #print(col,cols[i])
            same = y[col][(y[col]==1) & (y[cols[i]]==1)].sum()
            total =y[col][(y[col]==1)].sum()
            occur = round(same/total,2)
            cr.iloc[a,i]=occur 
    
    # create visuals
    
    graphs = [
        {
            'data': [
                Bar(
                    x=genre_names,
                    y=genre_counts
                )
            ],

            'layout': {
                'title': 'Distribution of Message Genres',
                'yaxis': {
                    'title': "Count"
                },
                'xaxis': {
                    'title': "Genre"
                }
            }
        },
        {
            'data': [
                Bar(
                    x=total_names,
                    y=total_counts
                )
            ],

            'layout': {
                'title': 'Distribution of Messages',
                'yaxis': {
                    'title': "Count"
                },
                'xaxis': {
                    'title': "Genre"
                }
            }
        },
        {
            'data': [
            Heatmap( z=cr.values.tolist(),
                   x= cols,
                   y= cols, colorscale='Viridis')
            ],

            'layout': {
                'title': 'Joint Probabilty Heatmap of Message Labels'
                }
        }        
        
    ]
    
    # encode plotly graphs in JSON
    ids = ["graph-{}".format(i) for i, _ in enumerate(graphs)]
    graphJSON = json.dumps(graphs, cls=plotly.utils.PlotlyJSONEncoder)
    
    # render web page with plotly graphs
    return render_template('master.html', ids=ids, graphJSON=graphJSON)


# web page that handles user query and displays model results
@app.route('/go')
def go():
    # save user input in query
    query = request.args.get('query', '') 

    # use model to predict classification for query
    classification_labels = model.predict([query])[0]
    classification_results = dict(zip(df.columns[5:], classification_labels))

    # This will render the go.html Please see that file. 
    return render_template(
        'go.html',
        query=query,
        classification_result=classification_results
    )


def main():
    #app.run(host='0.0.0.0', port=3001, debug=True,use_reloader=False)
    app.run(debug=True,use_reloader=False)
    #app.run(host='0.0.0.0', port=3001, debug=True)



/Users/duyminhpham/opt/anaconda3/envs/nbdev/lib/python3.7/site-packages/sklearn/base.py:338: UserWarning: Trying to unpickle estimator CountVectorizer from version 0.22.1 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  UserWarning,
/Users/duyminhpham/opt/anaconda3/envs/nbdev/lib/python3.7/site-packages/sklearn/base.py:338: UserWarning: Trying to unpickle estimator TfidfTransformer from version 0.22.1 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  UserWarning,
/Users/duyminhpham/opt/anaconda3/envs/nbdev/lib/python3.7/site-packages/sklearn/base.py:338: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from 

In [ ]:
main()

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


 * Running on http://127.0.0.1:5000 (Press CTRL+C to quit)
/Users/duyminhpham/opt/anaconda3/envs/nbdev/lib/python3.7/site-packages/ipykernel_launcher.py:31: RuntimeWarning: invalid value encountered in long_scalars
127.0.0.1 - - [07/Jul/2022 15:04:44] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [07/Jul/2022 15:04:44] "GET /favicon.ico HTTP/1.1" 404 -


In [ ]:
#export
if __name__ == '__main__':
    main()